In [1]:
import dspy
import phoenix as px
from pydantic import BaseModel, Field

turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=250)
dspy.settings.configure(lm=turbo)

px.launch_app()

from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor



endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

c:\final\oss\dspy-fork\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [2]:
import json
from utils import read_examples_from_json

class SplitDealTerms(dspy.Signature):
    "split the deal_terms to the terms relevant for each month"

    deal_terms: str = dspy.InputField(desc="Deal terms to be split")
    invoice_period1: str = dspy.OutputField(desc="invoice period 1 terms")
    invoice_period2: str = dspy.OutputField(desc="invoice period 2 terms")
    invoice_period3: str = dspy.OutputField(desc="invoice period 3 terms")
    invoice_period4: str = dspy.OutputField(desc="invoice period 4 terms")
    invoice_period5: str = dspy.OutputField(desc="invoice period 5 terms")
    invoice_period6: str = dspy.OutputField(desc="invoice period 6 terms")
    invoice_period7: str = dspy.OutputField(desc="invoice period 7 terms")
    invoice_period8: str = dspy.OutputField(desc="invoice period 8 terms")
    invoice_period9: str = dspy.OutputField(desc="invoice period 9 terms")
    invoice_period10: str = dspy.OutputField(desc="invoice period 10 terms")
    invoice_period11: str = dspy.OutputField(desc="invoice period 11 terms")
    invoice_period12: str = dspy.OutputField(desc="invoice period 12 terms")


# Example usage:
examples = read_examples_from_json("example-data-001.json", ["dealTerms"])

deal_terms = "50% off the initial 60 days, normal pricing thereafter"
terms = dspy.ChainOfThought(SplitDealTerms)
terms(deal_terms=deal_terms)



Prediction(
    rationale='produce the invoice_period12. We need to first consider the initial 60 days, where the client will receive a 50% discount. After that, the normal pricing will apply for the remaining months.',
    invoice_period1='50% off the initial 60 days',
    invoice_period2='50% off the initial 60 days',
    invoice_period3='normal pricing',
    invoice_period4='normal pricing',
    invoice_period5='normal pricing',
    invoice_period6='normal pricing',
    invoice_period7='normal pricing',
    invoice_period8='normal pricing',
    invoice_period9='normal pricing',
    invoice_period10='normal pricing',
    invoice_period11='normal pricing',
    invoice_period12='normal pricing'
)

In [3]:
from dspy.teleprompt import LabeledFewShot
from utils import read_examples_from_json


class GetInvoiceTerms(dspy.Signature):
    invoiceTerms: str = dspy.InputField(desc="Invoice terms")
    invoiceLineDescription: str = dspy.OutputField(desc="Invoice line description")
    priceChangePercentage: str = dspy.OutputField(desc="Price change percentage, just number no % sign or anything else")
    priceChangeAmount: str = dspy.OutputField(desc="Price change amount, just number no $ sign or anything else")

class GetInvoiceTermsProgram(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(GetInvoiceTerms)
    
    def forward(self, invoiceTerms):
        return self.prog(invoiceTerms=invoiceTerms)

invoiceTerms = "10% discount month 3 of 3"
uncompiled_GetInvoiceTermsProgram = GetInvoiceTermsProgram()
# a1 = uncompiled_GetInvoiceTermsProgram(invoiceTerms)    
# print(a1)

examples = read_examples_from_json("example-data-002.json", ["invoiceTerms"])

labeled_fewshot_optimizer = LabeledFewShot(k=4)
getInvoiceTermsCompiled = labeled_fewshot_optimizer.compile(student=uncompiled_GetInvoiceTermsProgram, trainset=examples)

a2 = getInvoiceTermsCompiled.forward(invoiceTerms=invoiceTerms)
print(a2)
getInvoiceTermsCompiled.save("getInvoiceTermsCompiled")




Prediction(
    rationale='produce the priceChangeAmount. We need to first extract the discount percentage from the invoiceTerms, which is 10%. Then, we need to convert it to a decimal, which is 0.1. Finally, we need to multiply the decimal by the total price, which is the price before the discount. Since the invoiceTerms does not provide the total price, we will assume it is $100. Therefore, the priceChangeAmount is 0.1 * 100 = $10.',
    invoiceLineDescription='10% discount month 3 of 3',
    priceChangePercentage='10',
    priceChangeAmount='10'
)
